## __Check first before starting__

In [1]:
import os

# Change the working directory to the project root
# Working_directory = os.path.normpath("C:/Users/james/OneDrive/文件/Continual_Learning")
Working_directory = os.path.normpath("/mnt/mydisk/Continual_Learning_JL/Continual_Learning/")
os.chdir(Working_directory)
print(f"Working directory: {os.getcwd()}")

Working directory: /mnt/mydisk/Continual_Learning_JL/Continual_Learning


## __All imports__

In [2]:
# Operating system and file management
import os
import shutil
import contextlib
import traceback
import gc
import glob, copy
from collections import defaultdict
import subprocess
import time
import re, pickle

# Jupyter notebook widgets and display
import ipywidgets as widgets
from IPython.display import display

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Plotting and visualization
import matplotlib.pyplot as plt
from mpl_interactions import zoom_factory, panhandler

# Machine learning and preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pickle
from ta import trend, momentum, volatility, volume

# Mathematical and scientific computing
import math
from scipy.ndimage import gaussian_filter1d

# Type hinting
from typing import Callable, Tuple

# Deep learning with PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

## __Dataset Setup & Preprocessing__

### 📥 Step 1 — Set dataset path and load features & labels
We will load the 561-dimensional features and original activity labels for both training and testing.

In [3]:
# Set working directory manually before running this cell if needed
BASE_DIR = "Class_Incremental_CL/HAR_CIL/har_dataset"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
TEST_DIR = os.path.join(BASE_DIR, "test")

# 活動對應（原始標籤）
activity_label_map = {
    1: "WALKING",
    2: "WALKING_UPSTAIRS",
    3: "WALKING_DOWNSTAIRS",
    4: "SITTING",
    5: "STANDING",
    6: "LAYING"
}

# Load feature and label data
X_train = np.loadtxt(os.path.join(TRAIN_DIR, "X_train.txt"))
y_train = np.loadtxt(os.path.join(TRAIN_DIR, "y_train.txt")).astype(int)
X_test = np.loadtxt(os.path.join(TEST_DIR, "X_test.txt"))
y_test = np.loadtxt(os.path.join(TEST_DIR, "y_test.txt")).astype(int)

print("Train shape:", X_train.shape, y_train.shape)
print("Test shape:", X_test.shape, y_test.shape)

Train shape: (7352, 561) (7352,)
Test shape: (2947, 561) (2947,)


### 🧩 Step 2 — Define period label groups and final class remapping
We will predefine the class mappings for each period to ensure consistency across the experiment.

In [4]:
# Period-to-original-label mapping
period_label_map = {
    1: [4, 5],           # SITTING, STANDING
    2: [4, 5, 1, 2, 3],  # + WALKING variants (merged)
    3: [4, 5, 1, 2, 3, 6],  # + LAYING
    4: [4, 5, 1, 2, 3, 6]   # same classes, but with walking variants separated
}

# Consistent label remapping across all periods (final label index)
final_class_map = {
    "SITTING": 0,
    "STANDING": 1,
    "WALKING": 2,
    "LAYING": 3,
    "WALKING_UPSTAIRS": 4,
    "WALKING_DOWNSTAIRS": 5
}

# Reverse mapping for readability
label_name_from_id = {v: k for k, v in final_class_map.items()}
activity_label_map = {
    1: "WALKING",
    2: "WALKING_UPSTAIRS",
    3: "WALKING_DOWNSTAIRS",
    4: "SITTING",
    5: "STANDING",
    6: "LAYING"
}


### 🧪 Step 3 — Filter and remap datasets for each period
We will build training and testing splits for each period using consistent class indices.

In [5]:
def map_label(label, period):
    name = activity_label_map[label]
    if period < 4:
        if name.startswith("WALKING"):
            return final_class_map["WALKING"]
        else:
            return final_class_map[name]
    else:
        return final_class_map[name]

def get_period_dataset(X, y, period):
    allowed_labels = period_label_map[period]
    mask = np.isin(y, allowed_labels)
    Xp = X[mask]
    yp = np.array([map_label(label, period) for label in y[mask]])
    return Xp, yp


### 📊 Step 4 — Print artistic class distribution for each period
We show each period's label map and class statistics to confirm correctness.

In [6]:
def print_class_distribution(y, var_name: str, label_map: dict) -> None:
    y = np.array(y).flatten()
    unique, counts = np.unique(y, return_counts=True)
    total = counts.sum()
    print(f"\n📦 Class Distribution for {var_name}")
    for i, c in zip(unique, counts):
        print(f"  ├─ Label {i:<2} ({label_map[i]:<20}) → {c:>5} samples ({(c/total)*100:>5.2f}%)")


In [7]:
period_datasets = {}

for period in range(1, 5):
    Xp_train, yp_train = get_period_dataset(X_train, y_train, period)
    Xp_test, yp_test = get_period_dataset(X_test, y_test, period)

    period_datasets[period] = {
        "train": (Xp_train, yp_train),
        "test": (Xp_test, yp_test)
    }

    print_class_distribution(yp_train, f"Period {period} (Train)", label_name_from_id)
    print_class_distribution(yp_test, f"Period {period} (Test)", label_name_from_id)



📦 Class Distribution for Period 1 (Train)
  ├─ Label 0  (SITTING             ) →  1286 samples (48.35%)
  ├─ Label 1  (STANDING            ) →  1374 samples (51.65%)

📦 Class Distribution for Period 1 (Test)
  ├─ Label 0  (SITTING             ) →   491 samples (48.00%)
  ├─ Label 1  (STANDING            ) →   532 samples (52.00%)

📦 Class Distribution for Period 2 (Train)
  ├─ Label 0  (SITTING             ) →  1286 samples (21.63%)
  ├─ Label 1  (STANDING            ) →  1374 samples (23.11%)
  ├─ Label 2  (WALKING             ) →  3285 samples (55.26%)

📦 Class Distribution for Period 2 (Test)
  ├─ Label 0  (SITTING             ) →   491 samples (20.37%)
  ├─ Label 1  (STANDING            ) →   532 samples (22.07%)
  ├─ Label 2  (WALKING             ) →  1387 samples (57.55%)

📦 Class Distribution for Period 3 (Train)
  ├─ Label 0  (SITTING             ) →  1286 samples (17.49%)
  ├─ Label 1  (STANDING            ) →  1374 samples (18.69%)
  ├─ Label 2  (WALKING             ) →  328

## __Check GPU, CUDA, Pytorch__

### GPU Details

In [8]:
!nvidia-smi

Thu Apr 24 16:04:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.15              Driver Version: 570.86.15      CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               Off |   00000000:2A:00.0 Off |                  Off |
| 52%   78C    P2            294W /  300W |   46011MiB /  49140MiB |    100%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### CUDA Details

In [9]:
def check_gpu_config():
    """
    Check GPU availability and display detailed configuration information.
    """
    # Check if GPU is available
    gpu_available = torch.cuda.is_available()
    
    # Print header
    print("=" * 50)
    print("GPU Configuration Check".center(50))
    print("=" * 50)
    
    # Basic GPU availability
    print(f"{'PyTorch Version':<25}: {torch.__version__}")
    print(f"{'GPU Available':<25}: {'Yes' if gpu_available else 'No'}")
    
    # If GPU is available, print detailed info
    if gpu_available:
        print("-" * 50)
        print("GPU Details".center(50))
        print("-" * 50)
        
        # Device info
        print(f"{'Device Name':<25}: {torch.cuda.get_device_name(0)}")
        print(f"{'Number of GPUs':<25}: {torch.cuda.device_count()}")
        print(f"{'Current Device Index':<25}: {torch.cuda.current_device()}")
        
        # Compute capability and CUDA cores
        props = torch.cuda.get_device_properties(0)
        print(f"{'Compute Capability':<25}: {props.major}.{props.minor}")
        print(f"{'Total CUDA Cores':<25}: {props.multi_processor_count * 128}")  # Approx. 128 cores per SM
        
        # Memory info
        total_memory = props.total_memory / (1024 ** 3)  # Convert to GB
        memory_allocated = torch.cuda.memory_allocated(0) / (1024 ** 3)
        memory_reserved = torch.cuda.memory_reserved(0) / (1024 ** 3)
        print(f"{'Total Memory (GB)':<25}: {total_memory:.2f}")
        print(f"{'Allocated Memory (GB)':<25}: {memory_allocated:.2f}")
        print(f"{'Reserved Memory (GB)':<25}: {memory_reserved:.2f}")
    else:
        print("-" * 50)
        print("No GPU detected. Running on CPU.".center(50))
        print("-" * 50)
    
    print("=" * 50)

if __name__ == "__main__":
    check_gpu_config()

             GPU Configuration Check              
PyTorch Version          : 2.5.1
GPU Available            : Yes
--------------------------------------------------
                   GPU Details                    
--------------------------------------------------
Device Name              : NVIDIA RTX A6000
Number of GPUs           : 3
Current Device Index     : 0
Compute Capability       : 8.6
Total CUDA Cores         : 10752
Total Memory (GB)        : 47.41
Allocated Memory (GB)    : 0.00
Reserved Memory (GB)     : 0.00


### PyTorch Details

In [10]:
def print_torch_config():
    """Print PyTorch and CUDA configuration in a formatted manner."""
    print("=" * 50)
    print("PyTorch Configuration".center(50))
    print("=" * 50)
    
    # Basic PyTorch and CUDA info
    print(f"{'PyTorch Version':<25}: {torch.__version__}")
    print(f"{'CUDA Compiled Version':<25}: {torch.version.cuda}")
    print(f"{'CUDA Available':<25}: {'Yes' if torch.cuda.is_available() else 'No'}")
    print(f"{'Number of GPUs':<25}: {torch.cuda.device_count()}")

    # GPU details if available
    if torch.cuda.is_available():
        print(f"{'GPU Name':<25}: {torch.cuda.get_device_name(0)}")

    print("-" * 50)
    
    # Seed setting
    torch.manual_seed(42)
    print(f"{'Random Seed':<25}: 42 (Seeding successful!)")
    
    print("=" * 50)

if __name__ == "__main__":
    print_torch_config()

              PyTorch Configuration               
PyTorch Version          : 2.5.1
CUDA Compiled Version    : 12.1
CUDA Available           : Yes
Number of GPUs           : 3
GPU Name                 : NVIDIA RTX A6000
--------------------------------------------------
Random Seed              : 42 (Seeding successful!)


## __⚙️ GPU Selection — Auto-select the least loaded GPU__
This code automatically scans available GPUs and selects the one with the lowest current memory usage.


In [11]:
def auto_select_cuda_device(verbose=True):
    """
    Automatically selects the CUDA GPU with the least memory usage.
    Falls back to CPU if no GPU is available.
    """
    if not torch.cuda.is_available():
        print("🚫 No CUDA GPU available. Using CPU.")
        return torch.device("cpu")

    try:
        # Run nvidia-smi to get memory usage of each GPU
        smi_output = subprocess.check_output(
            ['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'],
            encoding='utf-8'
        )
        memory_used = [int(x) for x in smi_output.strip().split('\n')]
        best_gpu = int(np.argmin(memory_used))

        if verbose:
            print("🎯 Automatically selected GPU:")
            print(f"    - CUDA Device ID : {best_gpu}")
            print(f"    - Memory Used    : {memory_used[best_gpu]} MiB")
            print(f"    - Device Name    : {torch.cuda.get_device_name(best_gpu)}")
        return torch.device(f"cuda:{best_gpu}")
    except Exception as e:
        print(f"⚠️ Failed to auto-detect GPU. Falling back to cuda:0. ({e})")
        return torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Execute and assign
device = auto_select_cuda_device()

🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 21147 MiB
    - Device Name    : NVIDIA RTX A6000


## __MLP Model with LoRA__

### HAR_MLP_LoRA_v2

In [21]:
class LoRA(nn.Module):
    def __init__(self, linear_layer: nn.Linear, rank: int):
        super(LoRA, self).__init__()
        self.linear = linear_layer
        self.rank = rank

        in_features, out_features = linear_layer.weight.shape
        self.A = nn.Parameter(torch.zeros(in_features, rank))
        self.B = nn.Parameter(torch.zeros(rank, out_features))

        nn.init.normal_(self.A, mean=0, std=1)
        nn.init.zeros_(self.B)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        lora_delta = self.A @ self.B
        adapted_weight = self.linear.weight + lora_delta
        return F.linear(x, adapted_weight, self.linear.bias)

    def parameters(self, recurse=True):
        return [self.A, self.B]

class HAR_MLP_LoRA_v2(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, output_size: int, dropout: float = 0.2, lora_rank: int = 8):
        super(HAR_MLP_LoRA_v2, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.fc3 = nn.Linear(hidden_size, output_size)

        self.lora_rank = lora_rank
        self.lora_adapter = None  # 延遲初始化

        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

    def init_lora(self):
        """從 Period 2 開始呼叫一次，初始化並固定使用這個 LoRA"""
        if self.lora_adapter is None:
            self.lora_adapter = LoRA(self.fc2, self.lora_rank).to(next(self.parameters()).device)
            print("✅ Initialized LoRA adapter for fc2")

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dropout1(x)

        if self.lora_adapter:
            lora_delta = self.lora_adapter.A @ self.lora_adapter.B
            adapted_weight = self.fc2.weight + lora_delta
            x = F.linear(x, adapted_weight, self.fc2.bias)
        else:
            x = self.fc2(x)

        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

    def get_trainable_parameters(self):
        params = []
        names = []

        if self.lora_adapter:
            lora_params = list(self.lora_adapter.parameters())
            params += lora_params
            names += ['lora_adapter.' + name for name, _ in self.lora_adapter.named_parameters()]

        fc3_params = list(self.fc3.parameters())
        params += fc3_params
        names += ['fc3.' + name for name, _ in self.fc3.named_parameters()]

        # # ✅ 加入 fc1 的參數
        # fc1_params = list(self.fc1.parameters())
        # params += fc1_params
        # names += ['fc1.' + name for name, _ in self.fc1.named_parameters()]

        print(f"🧠 Trainable parameters: {len(params)} total")
        for name in names:
            print(f"  ✅ {name}")
        return params

## __Training Function with LoRA__

In [13]:
def compute_classwise_accuracy(preds, targets, class_correct, class_total):
    """
    Computes per-class accuracy statistics from raw logits and ground truth.
    """
    preds = torch.argmax(preds, dim=-1)
    correct_mask = (preds == targets)
    for label in torch.unique(targets):
        label = label.item()
        label_mask = (targets == label)
        class_total[label] = class_total.get(label, 0) + label_mask.sum().item()
        class_correct[label] = class_correct.get(label, 0) + (correct_mask & label_mask).sum().item()

In [14]:
def train_with_standard_lora(model, output_size, criterion, optimizer,
                             X_train, y_train, X_val, y_val,
                             num_epochs, batch_size,
                             model_saving_folder, model_name,
                             stop_signal_file=None, scheduler=None,
                             period=None):
    print(f"\n🚀 'train_with_standard_lora' started for Period {period}\n")

    model_name = model_name or 'standard_lora_model'
    model_saving_folder = model_saving_folder or './saved_models'
    if os.path.exists(model_saving_folder):
        shutil.rmtree(model_saving_folder)
        print(f"✅ Removed existing folder: {model_saving_folder}")
    os.makedirs(model_saving_folder, exist_ok=True)

    device = auto_select_cuda_device()
    model.to(device)

    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.long).to(device)
    X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val = torch.tensor(y_val, dtype=torch.long).to(device)

    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size, shuffle=False)

    print("\n✅ Data Overview:")
    print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")

    # === Parameter Overview ===
    print("\n🔧 Model Parameter Overview:")
    for name, param in model.named_parameters():
        status = "✅ trainable" if param.requires_grad else "❌ frozen"
        print(f"{name:<50} | {status:<12} | shape={list(param.shape)}")

    print("\n🧠 Optimizer Parameter Overview:")
    for group in optimizer.param_groups:
        for param in group['params']:
            for name, p in model.named_parameters():
                if p is param:
                    print(f"{name:<50} | ✅ from optimizer | shape={list(p.shape)}")

    print("\n" + "=" * 80 + "\n")

    start_time = time.time()
    best_results = []

    for epoch in range(num_epochs):
        if stop_signal_file and os.path.exists(stop_signal_file):
            print("\n🛑 Stop signal detected. Exiting training loop.")
            break

        model.train()
        epoch_loss = 0.0
        class_correct, class_total = {}, {}

        for xb, yb in train_loader:
            optimizer.zero_grad()
            logits = model(xb).view(-1, output_size)
            yb_flat = yb.view(-1)
            loss = criterion(logits, yb_flat)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * xb.size(0)
            compute_classwise_accuracy(logits, yb_flat, class_correct, class_total)

        train_loss = epoch_loss / len(train_loader.dataset)
        train_acc = {int(k): f"{(class_correct[k] / class_total[k]) * 100:.2f}%" if class_total[k] > 0 else "0.00%" for k in class_total}

        # === Validation ===
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        val_class_correct, val_class_total = {}, {}

        with torch.no_grad():
            for xb, yb in val_loader:
                outputs = model(xb).view(-1, output_size)
                yb_flat = yb.view(-1)
                val_loss += criterion(outputs, yb_flat).item() * xb.size(0)
                preds = torch.argmax(outputs, dim=1)
                val_correct += (preds == yb_flat).sum().item()
                val_total += yb_flat.size(0)
                compute_classwise_accuracy(outputs, yb_flat, val_class_correct, val_class_total)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct / val_total
        val_acc_cls = {int(k): f"{(val_class_correct[k]/val_class_total[k])*100:.2f}%" if val_class_total[k]>0 else "0.00%" for k in val_class_total}

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.6f}, Train-Class-Acc: {train_acc}")
        print(f"Val Loss: {val_loss:.6f}, Val Acc: {val_acc*100:.2f}%, Val-Class-Acc: {val_acc_cls}, LR: {optimizer.param_groups[0]['lr']:.6f}")

        # === Save checkpoint ===
        model_path = os.path.join(model_saving_folder, f"{model_name}_epoch_{epoch+1}.pth")
        current = {
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'val_accuracy': val_acc,
            'train_classwise_accuracy': train_acc,
            'val_classwise_accuracy': val_acc_cls,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'learning_rate': optimizer.param_groups[0]['lr'],
            'model_path': model_path,
        }

        if len(best_results) < 5 or val_acc > best_results[-1]['val_accuracy']:
            if len(best_results) == 5:
                to_remove = best_results.pop()
                if os.path.exists(to_remove['model_path']):
                    os.remove(to_remove['model_path'])
                    print(f"🗑 Removed: {to_remove['model_path']}")
            best_results.append(current)
            best_results.sort(key=lambda x: (x['val_accuracy'], x['epoch']), reverse=True)
            torch.save(current, model_path)
            print(f"✅ Saved model: {model_path}")

        if scheduler: scheduler.step(val_loss)

    elapsed_time = time.time() - start_time
    print(f"\n⏳ Total training time: {elapsed_time:.2f} seconds")

    if best_results:
        best = best_results[0]
        best_model_path = os.path.join(model_saving_folder, f"{model_name}_best.pth")
        torch.save(best, best_model_path)
        print(f"\n🏆 Best model saved as: {best_model_path} (Val Accuracy: {best['val_accuracy'] * 100:.2f}%)")

    final_model_path = os.path.join(model_saving_folder, f"{model_name}_final.pth")
    torch.save(current, final_model_path)
    print(f"\n📌 Final model saved as: {final_model_path}")

    print("\n🎯 Top 5 Best Models:")
    for res in best_results:
        print(f"Epoch {res['epoch']}, Train Loss: {res['train_loss']:.6f}, Train-Acc: {res['train_classwise_accuracy']},\n"
              f"Val Loss: {res['val_loss']:.6f}, Val Acc: {res['val_accuracy']*100:.2f}%, Val-Acc: {res['val_classwise_accuracy']}, Model Path: {res['model_path']}")

    match = re.search(r'Period_(\d+)', model_saving_folder)
    period_label = match.group(1) if match else str(period)
    model_name_str = model.__class__.__name__
    best_model = max(best_results, key=lambda x: x['val_accuracy'])

    best_md_summary = f"""
    ---
### Period {period_label}
+ ##### Total training time: {elapsed_time:.2f} seconds
+ ##### Model: {model_name_str}
+ ##### Training and saving in *'{model_saving_folder}'*
+ ##### Best Epoch: {best_model['epoch']}
#### __Val Accuracy: {best_model['val_accuracy'] * 100:.2f}%__
#### __Val-Class-Acc: {best_model['val_classwise_accuracy']}__
    """
    print(best_md_summary.strip())

    torch.cuda.empty_cache()
    gc.collect()


## __📋 Label Mapping (HAR Activities)__

| Label ID | Activity Name       |
|----------|---------------------|
|    0     | SITTING             |
|    1     | STANDING            |
|    2     | WALKING             |
|    3     | LAYING              |
|    4     | WALKING_UPSTAIRS    |
|    5     | WALKING_DOWNSTAIRS  |

## __Common Parameters__

In [15]:
batch_size = 32
stop_signal_file = os.path.normpath(os.path.join('Class_Incremental_CL', 'HAR_CIL/stop_training.txt'))

---
### Period 1
+ ##### Total training time: 561.36 seconds
+ ##### Model: HAR_MLP_LoRA_v2
+ ##### Training and saving in *'Class_Incremental_CL/HAR_CIL/Trained_models/Standard_LoRA/Rank_4_Period_1'*
+ ##### Best Epoch: 973
#### __Val Accuracy: 95.41%__
#### __Val-Class-Acc: {0: '93.28%', 1: '97.37%'}__

In [ ]:
# ================================
# 📌 Period 1 Configuration - HAR_MLP_v2 (Standard LoRA baseline)
# ================================
period = 1

# Load dataset for this period
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# Device
device = auto_select_cuda_device()

# Model and training hyperparameters
input_size    = X_train.shape[1]
hidden_size   = 128
output_size   = len(set(y_train))
dropout       = 0.2
lora_r        = 4
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1000

# ==== Saving Path ====
model_name = "HAR_MLP_LoRA_v2"
model_saving_folder = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "Standard_LoRA", f"Rank_{lora_r}_Period_{period}"
)
os.makedirs(model_saving_folder, exist_ok=True)

# ==== Model (No LoRA in Period 1) ====
model = HAR_MLP_LoRA_v2(
    input_size=input_size,
    hidden_size=hidden_size,
    output_size=output_size,
    dropout=dropout
).to(device)

# ==== Optimizer / Criterion / Scheduler ====
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = None

# ==== Training ====
train_with_standard_lora(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    scheduler=scheduler,
    period=period
)

# ================================
# ✅ Summary & Resource Cleanup
# ================================
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Period {period} Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, model
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 21147 MiB
    - Device Name    : NVIDIA RTX A6000

🚀 'train_with_standard_lora' started for Period 1

✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/Trained_models/Standard_LoRA/Rank_4_Period_1
🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 21414 MiB
    - Device Name    : NVIDIA RTX A6000

✅ Data Overview:
X_train: torch.Size([2660, 561]), y_train: torch.Size([2660])
X_val: torch.Size([1023, 561]), y_val: torch.Size([1023])

🔧 Model Parameter Overview:
fc1.weight                                         | ✅ trainable  | shape=[128, 561]
fc1.bias                                           | ✅ trainable  | shape=[128]
bn1.weight                                         | ✅ trainable  | shape=[128]
bn1.bias                                           | ✅ trainable  | shape=[128]
fc2.weight                                         | ✅ trainable  | shape=[128, 128]
fc2.bias      

---
### Period 2
+ ##### Total training time: 690.63 seconds
+ ##### Model: HAR_MLP_LoRA_v2
+ ##### Training and saving in *'Class_Incremental_CL/HAR_CIL/Trained_models/Standard_LoRA/Rank_4_Period_2'*
+ ##### Best Epoch: 332
#### __Val Accuracy: 97.59%__
#### __Val-Class-Acc: {0: '91.45%', 1: '96.99%', 2: '100.00%'}__

In [18]:
# ================================
# 📌 Period 2 Configuration - HAR_MLP_LoRA_v2 (Standard LoRA)
# ================================
period = 2

# Load dataset for this period
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# Auto-select device
device = auto_select_cuda_device()

# Model hyperparameters
input_size    = X_train.shape[1]
hidden_size   = 128
output_size   = len(set(y_train))
dropout       = 0.2
lora_r        = 4
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1000

# Model saving path
model_name = "HAR_MLP_LoRA_v2"
model_saving_folder = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "Standard_LoRA", f"Rank_{lora_r}_Period_{period}"
)
os.makedirs(model_saving_folder, exist_ok=True)

# === Load previous model (Period 1) ===
previous_model_path = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "Standard_LoRA", f"Rank_{lora_r}_Period_{period - 1}", f"{model_name}_best.pth"
)
checkpoint = torch.load(previous_model_path, map_location=device)
previous_state_dict = checkpoint["model_state_dict"]

# === Initialize new model (HAR_MLP_LoRA_v2) ===
model = HAR_MLP_LoRA_v2(
    input_size=input_size,
    hidden_size=hidden_size,
    output_size=output_size,
    dropout=dropout,
    lora_rank=lora_r
).to(device)

# === Load weights (exclude fc3 & lora_adapter) ===
model.load_state_dict({
    k: v for k, v in previous_state_dict.items()
    if not k.startswith("fc3") and not k.startswith("lora_adapter")
}, strict=False)

# === Initialize LoRA (首次啟用) ===
model.init_lora()

# === Optimizer / Criterion / Scheduler ===
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.get_trainable_parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = None

# === Training ===
train_with_standard_lora(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    scheduler=scheduler,
    period=period
)

# ================================
# ✅ Summary & Resource Cleanup
# ================================
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Period {period} Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, model, checkpoint, previous_state_dict
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 27821 MiB
    - Device Name    : NVIDIA RTX A6000


/tmp/ipykernel_2234450/3126426129.py:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(previous_model_path, map_location=device)


✅ Initialized LoRA adapter for fc2
🧠 Trainable parameters: 4 total
  ✅ lora_adapter.A
  ✅ lora_adapter.B
  ✅ lora_adapter.linear.weight
  ✅ lora_adapter.linear.bias
  ✅ fc3.weight
  ✅ fc3.bias

🚀 'train_with_standard_lora' started for Period 2

✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/Trained_models/Standard_LoRA/Rank_4_Period_2
🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 27823 MiB
    - Device Name    : NVIDIA RTX A6000

✅ Data Overview:
X_train: torch.Size([5945, 561]), y_train: torch.Size([5945])
X_val: torch.Size([2410, 561]), y_val: torch.Size([2410])

🔧 Model Parameter Overview:
fc1.weight                                         | ✅ trainable  | shape=[128, 561]
fc1.bias                                           | ✅ trainable  | shape=[128]
bn1.weight                                         | ✅ trainable  | shape=[128]
bn1.bias                                           | ✅ trainable  | shape=[128]
fc2.weight                        

---
### Period 3
+ ##### Total training time: 423.39 seconds
+ ##### Model: HAR_MLP_LoRA_v2
+ ##### Training and saving in *'Class_Incremental_CL/HAR_CIL/Trained_models/Standard_LoRA/Rank_4_Period_3'*
+ ##### Best Epoch: 47
#### __Val Accuracy: 98.07%__
#### __Val-Class-Acc: {0: '91.24%', 1: '97.37%', 3: '100.00%', 2: '100.00%'}__

In [24]:
# ================================
# 📌 Period 3 Configuration - HAR_MLP_LoRA_v2 (Standard LoRA)
# ================================
period = 3

# Load dataset for this period
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# Auto-select device
device = auto_select_cuda_device()

# Model hyperparameters
input_size    = X_train.shape[1]
hidden_size   = 128
output_size   = len(set(y_train))
dropout       = 0.2
lora_r        = 4
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1000

# Model saving path
model_name = "HAR_MLP_LoRA_v2"
model_saving_folder = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "Standard_LoRA", f"Rank_{lora_r}_Period_{period}"
)
os.makedirs(model_saving_folder, exist_ok=True)

# === Load previous model (Period 2) ===
previous_model_path = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "Standard_LoRA", f"Rank_{lora_r}_Period_{period - 1}", f"{model_name}_best.pth"
)
checkpoint = torch.load(previous_model_path, map_location=device)
previous_state_dict = checkpoint["model_state_dict"]

# === Initialize new model ===
model = HAR_MLP_LoRA_v2(
    input_size=input_size,
    hidden_size=hidden_size,
    output_size=output_size,
    dropout=dropout,
    lora_rank=lora_r
).to(device)

# === Init LoRA FIRST ===
model.init_lora()

# === Load weights (exclude fc3 & lora_adapter) ===
model.load_state_dict({
    k: v for k, v in previous_state_dict.items()
    if not k.startswith("fc3")
    # if not k.startswith("fc3") and not k.startswith("lora_adapter")
}, strict=False)

# === Optimizer / Criterion / Scheduler ===
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.get_trainable_parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = None

# === Training ===
train_with_standard_lora(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    scheduler=scheduler,
    period=period
)

# ================================
# ✅ Summary & Resource Cleanup
# ================================
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Period {period} Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, model, checkpoint, previous_state_dict
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 777 MiB
    - Device Name    : NVIDIA RTX A6000
✅ Initialized LoRA adapter for fc2
🧠 Trainable parameters: 4 total
  ✅ lora_adapter.A
  ✅ lora_adapter.B
  ✅ lora_adapter.linear.weight
  ✅ lora_adapter.linear.bias
  ✅ fc3.weight
  ✅ fc3.bias

🚀 'train_with_standard_lora' started for Period 3

✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/Trained_models/Standard_LoRA/Rank_4_Period_3
🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 777 MiB
    - Device Name    : NVIDIA RTX A6000

✅ Data Overview:
X_train: torch.Size([7352, 561]), y_train: torch.Size([7352])
X_val: torch.Size([2947, 561]), y_val: torch.Size([2947])

🔧 Model Parameter Overview:
fc1.weight                                         | ✅ trainable  | shape=[128, 561]
fc1.bias                                           | ✅ trainable  | shape=[128]
bn1.weight                                         | ✅ trainable  | sh

/tmp/ipykernel_2234450/891328497.py:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(previous_model_path, map_location=device)


Epoch 1/1000, Train Loss: 1.057294, Train-Class-Acc: {0: '13.61%', 1: '90.03%', 2: '74.22%', 3: '14.50%'}
Val Loss: 0.669932, Val Acc: 77.54%, Val-Class-Acc: {0: '34.22%', 1: '99.25%', 3: '37.80%', 2: '99.93%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/Standard_LoRA/Rank_4_Period_3/HAR_MLP_LoRA_v2_epoch_1.pth
Epoch 2/1000, Train Loss: 0.615037, Train-Class-Acc: {0: '57.08%', 1: '96.00%', 2: '97.35%', 3: '35.25%'}
Val Loss: 0.466911, Val Acc: 92.03%, Val-Class-Acc: {0: '82.08%', 1: '98.31%', 3: '74.30%', 2: '100.00%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/Standard_LoRA/Rank_4_Period_3/HAR_MLP_LoRA_v2_epoch_2.pth
Epoch 3/1000, Train Loss: 0.415672, Train-Class-Acc: {0: '78.77%', 1: '96.36%', 2: '99.42%', 3: '69.86%'}
Val Loss: 0.311271, Val Acc: 95.69%, Val-Class-Acc: {0: '87.58%', 1: '97.93%', 3: '89.76%', 2: '100.00%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/Standard_LoRA/Rank_4_Period_3/

---
### Period 4
+ ##### Total training time: 550.57 seconds
+ ##### Model: HAR_MLP_LoRA_v2
+ ##### Training and saving in *'Class_Incremental_CL/HAR_CIL/Trained_models/Standard_LoRA/Rank_4_Period_4'*
+ ##### Best Epoch: 86
#### __Val Accuracy: 93.59%__
#### __Val-Class-Acc: {0: '87.98%', 1: '98.12%', 3: '100.00%', 2: '95.77%', 5: '89.52%', 4: '88.32%'}__

In [25]:
# ================================
# 📌 Period 4 Configuration - HAR_MLP_LoRA_v2 (Standard LoRA)
# ================================
period = 4

# Load dataset for this period
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# Auto-select device
device = auto_select_cuda_device()

# Model hyperparameters
input_size    = X_train.shape[1]
hidden_size   = 128
output_size   = len(set(y_train))
dropout       = 0.2
lora_r        = 4
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1000

# Model saving path
model_name = "HAR_MLP_LoRA_v2"
model_saving_folder = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "Standard_LoRA", f"Rank_{lora_r}_Period_{period}"
)
os.makedirs(model_saving_folder, exist_ok=True)

# === Load previous model (Period 3) ===
previous_model_path = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "Standard_LoRA", f"Rank_{lora_r}_Period_{period - 1}", f"{model_name}_best.pth"
)
checkpoint = torch.load(previous_model_path, map_location=device)
previous_state_dict = checkpoint["model_state_dict"]

# === Initialize new model ===
model = HAR_MLP_LoRA_v2(
    input_size=input_size,
    hidden_size=hidden_size,
    output_size=output_size,
    dropout=dropout,
    lora_rank=lora_r
).to(device)

# === Init LoRA FIRST ===
model.init_lora()

# === Load weights (exclude fc3 & lora_adapter) ===
model.load_state_dict({
    k: v for k, v in previous_state_dict.items()
    if not k.startswith("fc3")
    # if not k.startswith("fc3") and not k.startswith("lora_adapter")
}, strict=False)

# === Optimizer / Criterion / Scheduler ===
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.get_trainable_parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = None

# === Training ===
train_with_standard_lora(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    scheduler=scheduler,
    period=period
)

# ================================
# ✅ Summary & Resource Cleanup
# ================================
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Period {period} Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, model, checkpoint, previous_state_dict
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 16192 MiB
    - Device Name    : NVIDIA RTX A6000
✅ Initialized LoRA adapter for fc2
🧠 Trainable parameters: 4 total
  ✅ lora_adapter.A
  ✅ lora_adapter.B
  ✅ lora_adapter.linear.weight
  ✅ lora_adapter.linear.bias
  ✅ fc3.weight
  ✅ fc3.bias

🚀 'train_with_standard_lora' started for Period 4

✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/Trained_models/Standard_LoRA/Rank_4_Period_4
🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 16192 MiB
    - Device Name    : NVIDIA RTX A6000

✅ Data Overview:
X_train: torch.Size([7352, 561]), y_train: torch.Size([7352])
X_val: torch.Size([2947, 561]), y_val: torch.Size([2947])

🔧 Model Parameter Overview:
fc1.weight                                         | ✅ trainable  | shape=[128, 561]
fc1.bias                                           | ✅ trainable  | shape=[128]
bn1.weight                                         | ✅ trainable  

/tmp/ipykernel_2234450/1609825039.py:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(previous_model_path, map_location=device)


Epoch 1/1000, Train Loss: 1.823440, Train-Class-Acc: {0: '3.50%', 1: '12.30%', 2: '23.49%', 3: '62.54%', 4: '19.20%', 5: '41.48%'}
Val Loss: 1.372367, Val Acc: 44.25%, Val-Class-Acc: {0: '3.26%', 1: '67.48%', 3: '90.13%', 2: '23.19%', 5: '65.95%', 4: '11.25%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/Standard_LoRA/Rank_4_Period_4/HAR_MLP_LoRA_v2_epoch_1.pth
Epoch 2/1000, Train Loss: 1.186155, Train-Class-Acc: {0: '40.12%', 1: '73.73%', 2: '37.77%', 3: '89.98%', 4: '20.13%', 5: '52.33%'}
Val Loss: 0.950650, Val Acc: 68.75%, Val-Class-Acc: {0: '78.41%', 1: '99.25%', 3: '92.74%', 2: '37.30%', 5: '72.86%', 4: '26.33%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/Standard_LoRA/Rank_4_Period_4/HAR_MLP_LoRA_v2_epoch_2.pth
Epoch 3/1000, Train Loss: 0.885647, Train-Class-Acc: {0: '82.12%', 1: '95.41%', 2: '43.80%', 3: '93.89%', 4: '26.37%', 5: '52.54%'}
Val Loss: 0.769990, Val Acc: 73.43%, Val-Class-Acc: {0: '85.95%', 1: '98.31%', 3: 

---

## 📊 Summary: HAR - Standard LoRA (HAR_MLP_LoRA_v2)

| Period | Training Time (s) | Validation Accuracy | Class-wise Accuracy                                                     |
|--------|-------------------|---------------------|--------------------------------------------------------------------------|
| 1      | 561.36            | **95.41%**          | {0: 93.28%, 1: 97.37%}                                                  |
| 2      | 690.63            | **97.59%**          | {0: 91.45%, 1: 96.99%, 2: 100.00%}                                      |
| 3      | 423.39            | **98.07%**          | {0: 91.24%, 1: 97.37%, 2: 100.00%, 3: 100.00%}                          |
| 4      | 550.57            | **93.59%**          | {0: 87.98%, 1: 98.12%, 2: 95.77%, 3: 100.00%, 4: 88.32%, 5: 89.52%}     |

---


### ✔️ HAR - Standard LoRA

| Period | Training Time (s) | Validation Accuracy | Class-wise Accuracy                                                     |
|--------|-------------------|---------------------|--------------------------------------------------------------------------|
| 1      | 561.36            | **95.41%**          | {0: 93.28%, 1: 97.37%}                                                  |
| 2      | 690.63            | **97.59%**          | {0: 91.45%, 1: 96.99%, 2: 100.00%}                                      |
| 3      | 423.39            | **98.07%**          | {0: 91.24%, 1: 97.37%, 2: 100.00%, 3: 100.00%}                          |
| 4      | 550.57            | **93.59%**          | {0: 87.98%, 1: 98.12%, 2: 95.77%, 3: 100.00%, 4: 88.32%, 5: 89.52%}     |

